**Import required libraries and scripts**

In [ ]:
import os
for dir in os.listdir('/media/mario/T7/FINISHED/FINISHED2/'):
    print(dir)

In [ ]:
import os
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.performance_calculation import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

for dir in os.listdir('/media/mario/T7/FINISHED/FINISHED2/'):
    print(dir)
    software = '/home/mario/DockM8/software'
    protein_file = f'/media/mario/T7/FINISHED/FINISHED2/{dir}/receptor_protoss_prepared.pdb'
    ref_file = f'/media/mario/T7/FINISHED/FINISHED2/{dir}/crystal_ligand_protoss.sdf'
    pocket = 'reference'
    protonation = 'pkasolver'
    docking_library = f'/media/mario/T7/FINISHED/FINISHED2/{dir}/merged_actives_decoys.sdf'
    docking_programs = ['GNINA', 'SMINA', 'PLANTS']
    clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
    clustering_method = 'KMedoids'
    rescoring= ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'SCORCH', 'RTMScore', 'vinardo']
    id_column = 'ID'
    n_poses = 10
    exhaustiveness = 8
    parallel = 1
    ncpus = int(os.cpu_count()/2)
    #Create a temporary folder for all further calculations
    w_dir = os.path.dirname(protein_file)
    print('The working directory has been set to:', w_dir)
    create_temp_folder(w_dir+'/temp')
    #try:   
    for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring, ncpus)
    calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)
    apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)
    #except Exception as e:
    #    print ('failed for '+dir)
    #    print(e)


In [ ]:
def rename_columns_in_csv(root_dir, target_file, col_rename_dict):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)

                # Read the CSV file
                df = pd.read_csv(filepath)

                # Rename columns according to the dictionary
                df.rename(columns=col_rename_dict, inplace=True)

                # Write the modified CSV file back to the same location
                df.to_csv(filepath, index=False)
                print(f"Columns renamed in: {filepath}")

root_directory = "/media/mario/T7/FINISHED/FINISHED2/"
target_csv_file = "allposes_rescored.csv"
column_rename_dict = {
    "AD4_Affinity": "AD4",
    "Vinardo_Affinity": "Vinardo",
    "GNINA_Affinity": "GNINA",
    "GNINA_CNN_Score": "CNN-Score",
    "GNINA_CNN_Affinity": "CNN-Affinity",
    'LinF9_Affinity':'LinF9',
    'SCORCH_pose_score':'SCORCH',
    # Add more column names to be renamed here
}
rename_columns_in_csv(root_directory, target_csv_file, column_rename_dict)

In [ ]:
import os
import pandas as pd

def rename_and_merge_csv_files(root_dir, target_file, output_file):
    merged_df = None

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, index_col=0)
                print(df)

                dirname = os.path.basename(dirpath)
                print(dirpath.replace('/media/mario/T7/FINISHED/FINISHED2/', '').replace('/temp/consensus', ''))
                df.rename(columns={"EF1%": dirpath.replace('/media/mario/T7/FINISHED/FINISHED2/', '').replace('/temp/consensus', '')}, inplace=True)
                df.drop(columns='EF10%', inplace=True)
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on=["Scoring Function", "Clustering Metric"])

    if merged_df is not None:
        merged_df.to_csv(output_file, index=False)
        print(f"Merged CSV file saved to: {output_file}")


root_directory = "/media/mario/T7/FINISHED/FINISHED2/"
target_csv_file = "EF_single_functions.csv"
output_csv_file = "merged_output.csv"

rename_and_merge_csv_files(root_directory, target_csv_file, output_csv_file)


In [4]:
import os
import pandas as pd

def rename_and_merge_csv_files(root_dir, target_file, output_file):
    merged_df = None

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, index_col=0)

                dirname = os.path.basename(dirpath)
                print(dirpath.replace('/media/mario/T7/FINISHED/FINISHED2/', '').replace('/temp/consensus', ''))
                df.rename(columns={"EF1%": dirpath.replace('/media/mario/T7/FINISHED/FINISHED2/', '').replace('/temp/consensus', '')}, inplace=True)
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on=["clustering_method", "selected_columns", 'method_name'])

    if merged_df is not None:
        merged_df.to_csv(output_file, index=False)
        print(f"Merged CSV file saved to: {output_file}")


root_directory = "/media/mario/T7/FINISHED/FINISHED2/"
target_csv_file = "consensus_summary.csv"
output_csv_file = "merged_output_consensus.csv"

rename_and_merge_csv_files(root_directory, target_csv_file, output_csv_file)


hs90a
hxk4
igf1r
kith
kpcb
lfa1
lkha4
nram
prgr
ptn1
pyrd
reni
urok
xiap
Merged CSV file saved to: merged_output_consensus.csv
